In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
from tqdm import tqdm
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Load Dataset

image_dir = "celebA/celeba/img_align_celeba"
os.makedirs("test", exist_ok=True)

transform = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

/sciclone/data10/iahewababarand/genVision-celebA/gen-env/lib64/python3.11/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3, features_g=64):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, img_channels=3, features_d=64):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, features_d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d, features_d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 2, features_d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 4, features_d * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.net(img).view(-1)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
z_dim = 100
lr = 2e-4
n_epochs = 10
batch_size = 64

G = Generator(z_dim).to(device)
D = Discriminator().to(device)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)

cuda


In [5]:
# Training Loop
for epoch in range(n_epochs):
    g_loss_epoch = 0.0
    d_loss_epoch = 0.0
    num_batches = 0

    for real_imgs, _ in tqdm(dataloader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)

        # Train Discriminator
        z = torch.randn(batch_size, z_dim, 1, 1, device=device)
        fake_imgs = G(z)

        real_labels = torch.ones(batch_size, device=device)
        fake_labels = torch.zeros(batch_size, device=device)

        output_real = D(real_imgs)
        output_fake = D(fake_imgs.detach())
        loss_D = criterion(output_real, real_labels) + criterion(output_fake, fake_labels)

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        output_fake = D(fake_imgs)
        loss_G = criterion(output_fake, real_labels)

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        g_loss_epoch += loss_G.item()
        d_loss_epoch += loss_D.item()
        num_batches += 1

    # Save sample images
    with torch.no_grad():
        samples = G(fixed_noise)
        save_image(samples, f"gan_outputs/epoch_{epoch+1:03d}.png", normalize=True)

    # Print average losses
    print(f"Epoch [{epoch+1}/{n_epochs}]  Loss_D: {d_loss_epoch/num_batches:.4f}  Loss_G: {g_loss_epoch/num_batches:.4f}")


Epoch 1/10:   0%|          | 0/3166 [00:00<?, ?it/s]

Epoch 1/10:   0%|          | 1/3166 [00:01<59:18,  1.12s/it]

Epoch 1/10:   0%|          | 4/3166 [00:01<12:42,  4.15it/s]

Epoch 1/10:   0%|          | 7/3166 [00:01<07:08,  7.38it/s]

Epoch 1/10:   0%|          | 10/3166 [00:01<05:01, 10.48it/s]

...

Epoch 10/10: 100%|█████████▉| 3154/3166 [03:09<00:00, 16.51it/s]

Epoch 10/10: 100%|█████████▉| 3158/3166 [03:10<00:00, 16.57it/s]

Epoch 10/10: 100%|█████████▉| 3162/3166 [03:10<00:00, 19.29it/s]

Epoch 10/10: 100%|██████████| 3166/3166 [03:10<00:00, 16.63it/s]

Epoch [10/10]  Loss_D: 0.2338  Loss_G: 5.3244


In [6]:
os.makedirs("gan_outputs/generated", exist_ok=True)
G.eval()
with torch.no_grad():
    for i in tqdm(range(0, 10000, 64), desc="Generating final images"):
        z = torch.randn(64, z_dim, 1, 1, device=device)
        gen_imgs = G(z)
        for j in range(gen_imgs.size(0)):
            save_image(gen_imgs[j], f"gan_outputs/generated/{i + j:05d}.png", normalize=True)


Generating final images:   0%|          | 0/157 [00:00<?, ?it/s]

Generating final images:   1%|          | 1/157 [00:00<00:47,  3.28it/s]

Generating final images:   1%|▏         | 2/157 [00:00<00:35,  4.33it/s]

Generating final images:   2%|▏         | 3/157 [00:00<00:32,  4.70it/s]

Generating final images:   3%|▎         | 4/157 [00:00<00:31,  4.92it/s]

...

Generating final images:  98%|█████████▊| 154/157 [00:28<00:00,  5.45it/s]

Generating final images:  99%|█████████▊| 155/157 [00:28<00:00,  5.47it/s]

Generating final images:  99%|█████████▉| 156/157 [00:28<00:00,  5.44it/s]

Generating final images: 100%|██████████| 157/157 [00:28<00:00,  5.51it/s]

Generating final images: 100%|██████████| 157/157 [00:28<00:00,  5.43it/s]